# 1-Bar+Line Chart + Check Every 100 Rows

### Overall, the script enables the user to visualize and explore the data interactively, displaying a subset of rows at a time, and allowing for easy navigation through the dataset

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display, clear_output

# Load your CSV file into a DataFrame
df = pd.read_csv(r"BTC-Dataset-USDM-2024-04-01 - 2024-04-17 - 5M.csv")
df.set_index('date_time', inplace=True)

# Define bins and labels
bins = [0, 100000, 200000, 300000, 500000, 800000, 1000000, 2000000, 5000000, 10000000, 10000000000000]
labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Create the new column 'liq_range' based on the 'original_volume' column
df['liq_range'] = pd.cut(df['original_volume'], bins=bins, labels=labels, include_lowest=True)

# Initialize start index
start_index = 0

while True:
    # Prompt the user for input
    user_input = input("Press any key to show the next 100 rows, or 'q' to quit: ")
    
    # Check if user wants to quit
    if user_input.lower() == 'q':
        break
    
    # Update end index
    end_index = start_index + 400
    
    # Select the next 100 rows
    df_subset = df.iloc[start_index:end_index]

    clear_output(wait=True)
    # Combine the plots
    fig = go.Figure()

    # Add trace for Volume_True and Volume_False as line plots
    trace_volume_true = go.Scatter(x=df_subset.index, y=df_subset['volume_true'], mode='lines', name='Volume_True', line=dict(color='rgba(0, 0, 255, 0.25)'))  # Blue with fade
    trace_volume_false = go.Scatter(x=df_subset.index, y=df_subset['volume_false'], mode='lines', name='Volume_False', line=dict(color='rgba(255, 165, 0, 0.25)'))  # Orange with fade

    # Add trace for Vdiff(False-True) as a bar chart with custom colors
    bar_colors = ['red' if diff < 0 else 'green' for diff in df_subset['vdiff(false-true)'].fillna(0)]
    trace_vdiff = go.Bar(x=df_subset.index, y=df_subset['vdiff(false-true)'], name='Vdiff', marker=dict(color=bar_colors))

    # Add traces to the figure
    fig.add_trace(trace_volume_true)
    fig.add_trace(trace_volume_false)
    fig.add_trace(trace_vdiff)

    # Add vertical lines for labels 1 to 9 with different colors
    colors = ['yellow','orange','red','purple', 'blue', 'black', 'brown', 'green']
    labels_text = ['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5', 'Level 6', 'Level 7', 'Level 8', 'Level 9']
    for label, color, text in zip(range(1, 10), colors, labels_text):
        df_filtered = df_subset[df_subset['liq_range'] == label]
        print(f"Label {label}: {df_filtered.shape[0]} points found")
        for date_time in df_filtered.index:
            # Add vertical lines
            fig.add_shape(
                type="line",
                x0=date_time,
                y0=min(df_subset['vdiff(false-true)']),  # Set y0 to the minimum of vdiff column
                x1=date_time,
                y1=max(df_subset['vdiff(false-true)']),  # Set y1 to the maximum of vdiff column
                line=dict(
                    color=color,
                    width=2,
                    dash="dashdot",
                ),
            )
            # Format the numeric values for hover information
            volume_value = "{:,.0f}".format(df_filtered.loc[date_time, 'original_volume']).split('.')[0]  # Remove everything after the dot and add commas every 3 digits
            volume_true = "{:,.0f}".format(df_filtered.loc[date_time, 'volume_true']).split('.')[0]
            volume_false = "{:,.0f}".format(df_filtered.loc[date_time, 'volume_false']).split('.')[0]
            vdiff_value = "{:,.0f}".format(df_filtered.loc[date_time, 'vdiff(false-true)']).split('.')[0]
            # Add text annotation for hover information
            hover_text = f"Date: {date_time}<br>Liquidity Level: {text}<br>Original Volume: {volume_value}"
            fig.add_trace(go.Scatter(x=[date_time], y=[0], mode='markers', marker=dict(size=8, color=color), text=hover_text, hoverinfo='text', showlegend=False))

    # Check if the sum of volume_true and volume_false exceeds 100 million, and add a scatter point with adjustable size
    df_subset['total_volume'] = df_subset['volume_true'] + df_subset['volume_false']
    if df_subset['total_volume'].max() > 100000000:
        df_filtered = df_subset[df_subset['total_volume'] > 100000000]
        for date_time in df_filtered.index:
            total_volume_formatted = "{:,.0f}".format(df_filtered.loc[date_time, 'total_volume'])
            hover_text = f"Date: {date_time}<br>Total Volume: {total_volume_formatted}"
            fig.add_trace(go.Scatter(x=[date_time], y=[0], mode='markers', marker=dict(size=15, color='black'), text=hover_text, hoverinfo='text', showlegend=False))

    # Set the layout
    fig.update_layout(
        title="Volume Comparison and Liquidity Ranges",
        xaxis_title="Date",
        yaxis_title="Volume",
        hovermode="x unified",
        width=1850,  # Specify the width of the figure
        height=950,  # Specify the height of the figure
        #template="plotly_white",  # Change this to the desired template name
        yaxis_range=[min(df_subset['volume_true']), max(df_subset['volume_false'])]  # Set y-axis range
    )

    # Show the plot
    fig.show()
    
    # Update start index for the next iteration
    start_index = end_index



Label 1: 0 points found
Label 2: 1 points found
Label 3: 0 points found
Label 4: 0 points found
Label 5: 0 points found
Label 6: 2 points found
Label 7: 3 points found
Label 8: 5 points found


C:\Users\Mcc\AppData\Local\Temp\ipykernel_359108\1798091333.py:80: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# 2-Bar+Line Chart

### Overall, the script visualizes the volume comparison and liquidity ranges for a given dataset using Plotly, providing insights into trading aggregation volumes and liquidity levels over time

In [19]:
import pandas as pd
import plotly.graph_objects as go

# Load your CSV file into a DataFrame
df = pd.read_csv(r"C:\Users\Mcc\Desktop\Data\Historical Data\.All in One\BTC-Dataset-USDM-2024-04-01 - 2024-04-17 - 5M.csv")
df.set_index('date_time', inplace=True)

df = df.iloc[0:100]
print(df.shape[0])

# Define bins and labels
bins = [0, 100000, 200000, 300000, 500000, 800000, 1000000, 2000000, 5000000, 10000000, 10000000000000]
labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Create the new column 'liq_range' based on the 'original_volume' column
df['liq_range'] = pd.cut(df['original_volume'], bins=bins, labels=labels, include_lowest=True)

# Combine the plots
fig = go.Figure()

# Add trace for Volume_True and Volume_False as line plots
trace_volume_true = go.Scatter(x=df.index, y=df['volume_true'], mode='lines', name='Volume_True', line=dict(color='rgba(0, 0, 255, 0.25)'))  # Blue with fade
trace_volume_false = go.Scatter(x=df.index, y=df['volume_false'], mode='lines', name='Volume_False', line=dict(color='rgba(255, 165, 0, 0.25)'))  # Orange with fade

# Add trace for Vdiff(False-True) as a bar chart with custom colors
bar_colors = ['red' if diff < 0 else 'green' for diff in df['vdiff(false-true)'].fillna(0)]
trace_vdiff = go.Bar(x=df.index, y=df['vdiff(false-true)'], name='Vdiff', marker=dict(color=bar_colors))

# Add traces to the figure
fig.add_trace(trace_volume_true)
fig.add_trace(trace_volume_false)
fig.add_trace(trace_vdiff)

# Add vertical lines for labels 1 to 9 with different colors
colors = ['yellow','orange','red','purple', 'blue', 'black', 'brown', 'green']
labels_text = ['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5', 'Level 6', 'Level 7', 'Level 8', 'Level 9']
for label, color, text in zip(range(1, 10), colors, labels_text):
    df_filtered = df[df['liq_range'] == label]
    print(f"Label {label}: {df_filtered.shape[0]} points found")
    for date_time in df_filtered.index:
        # Add vertical lines
        fig.add_shape(
            type="line",
            x0=date_time,
            y0=min(df['vdiff(false-true)']),  # Set y0 to the minimum of vdiff column
            x1=date_time,
            y1=max(df['vdiff(false-true)']),  # Set y1 to the maximum of vdiff column
            line=dict(
                color=color,
                width=2,
                dash="dashdot",
            ),
        )
        # Format the numeric values for hover information
        volume_value = "{:,.0f}".format(df_filtered.loc[date_time, 'original_volume']).split('.')[0]  # Remove everything after the dot and add commas every 3 digits
        volume_true = "{:,.0f}".format(df_filtered.loc[date_time, 'volume_true']).split('.')[0]
        volume_false = "{:,.0f}".format(df_filtered.loc[date_time, 'volume_false']).split('.')[0]
        vdiff_value = "{:,.0f}".format(df_filtered.loc[date_time, 'vdiff(false-true)']).split('.')[0]
        # Add text annotation for hover information
        hover_text = f"Date: {date_time}<br>Liquidity Level: {text}<br>Original Volume: {volume_value}"
        fig.add_trace(go.Scatter(x=[date_time], y=[0], mode='markers', marker=dict(size=8, color=color), text=hover_text, hoverinfo='text', showlegend=False))

# Check if the sum of volume_true and volume_false exceeds 100 million, and add a scatter point with adjustable size
df['total_volume'] = df['volume_true'] + df['volume_false']
if df['total_volume'].max() > 100000000:
    df_filtered = df[df['total_volume'] > 100000000]
    for date_time in df_filtered.index:
        total_volume_formatted = "{:,.0f}".format(df_filtered.loc[date_time, 'total_volume'])
        hover_text = f"Date: {date_time}<br>Total Volume: {total_volume_formatted}"
        fig.add_trace(go.Scatter(x=[date_time], y=[0], mode='markers', marker=dict(size=15, color='black'), text=hover_text, hoverinfo='text', showlegend=False))


# Set the layout
fig.update_layout(
    title="Volume Comparison and Liquidity Ranges",
    xaxis_title="Date",
    yaxis_title="Volume",
    hovermode="x unified",
    width=1850,  # Specify the width of the figure
    height=950,  # Specify the height of the figure
    #template="plotly_white",  # Change this to the desired template name
    yaxis_range=[min(df['volume_true']), max(df['volume_false'])]  # Set y-axis range
)

# Show the plot
fig.show()


400
Label 1: 5 points found
Label 2: 2 points found
Label 3: 2 points found
Label 4: 4 points found
Label 5: 1 points found
Label 6: 4 points found
Label 7: 10 points found
Label 8: 12 points found


# 3-Candlestick Version

### Overall, the script loads data from a CSV file, processes it, creates a candlestick chart with additional annotations for liquidation levels and total volume, and then displays the chart using Plotly.

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Load your CSV file into a DataFrame
df = pd.read_csv(r"C:\Users\Mcc\Desktop\Data\Historical Data\.All in One\all_in_one_5M.csv")
df['total_volume'] = df['volume_true'] + df['volume_false']
df['close_shifted'] = df['vdiff(false-true)'].shift(1)
df = df.iloc[-400:]
df.set_index('date_time', inplace=True)
df['volume_true'] = df['volume_true'] * -1

# Define the bins and labels for the ranges
bins = [0, 100000, 200000, 300000, 500000, 800000, 1000000, 2000000, 5000000, 10000000, 10000000000000]
labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
# Create the new column 'liq_range' based on the 'original_volume' column
df['liq_range'] = pd.cut(df['original_volume'], bins=bins, labels=labels, include_lowest=True)

# Create the candlestick chart
fig = go.Figure(data=[go.Candlestick(
    x=df.index,
    close=df['vdiff(false-true)'],
    open=df['close_shifted'],
    high=df['volume_false'],
    low=df['volume_true'],
    increasing_line_color='green',  # Set the color for increasing candles
    decreasing_line_color='red'  # Set the color for decreasing candles
)])

# Add hover information
fig.update_traces(
    hoverinfo="x+y+text"
)

# Add vertical lines for liquidation levels
colors = ['yellow','orange','red','purple', 'blue', 'black', 'brown', 'green']
labels_text = ['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5', 'Level 6', 'Level 7', 'Level 8', 'Level 9']
for label, color, text in zip(range(1, 10), colors, labels_text):
    df_filtered = df[df['liq_range'] == label]
    print(f"Label {label}: {df_filtered.shape[0]} points found")
    for date_time in df_filtered.index:
        # Add vertical lines
        fig.add_shape(
            type="line",
            x0=date_time,
            y0=min(df['vdiff(false-true)']),  # Set y0 to the minimum of vdiff column
            x1=date_time,
            y1=max(df['vdiff(false-true)']),  # Set y1 to the maximum of vdiff column
            line=dict(
                color=color,
                width=2,
                dash="dashdot",
            ),
        )
        # Format the numeric values for hover information
        volume_value = "{:,.0f}".format(df_filtered.loc[date_time, 'original_volume']).split('.')[0]  # Remove everything after the dot and add commas every 3 digits
        volume_true = "{:,.0f}".format(df_filtered.loc[date_time, 'volume_true']).split('.')[0]
        volume_false = "{:,.0f}".format(df_filtered.loc[date_time, 'volume_false']).split('.')[0]
        vdiff_value = "{:,.0f}".format(df_filtered.loc[date_time, 'vdiff(false-true)']).split('.')[0]
        # Add text annotation for hover information
        hover_text = f"Date: {date_time}<br>Liquidity Level: {text}<br>Original Volume: {volume_value}"
        fig.add_trace(go.Scatter(x=[date_time], y=[0], mode='markers', marker=dict(size=8, color=color), text=hover_text, hoverinfo='text', showlegend=False))

# Add scatter plot for total volume
if df['total_volume'].max() > 100000000:
    df_filtered = df[df['total_volume'] > 100000000]
    for date_time in df_filtered.index:
        total_volume_formatted = "{:,.0f}".format(df_filtered.loc[date_time, 'total_volume'])
        hover_text = f"Date: {date_time}<br>Total Volume: {total_volume_formatted}"
        fig.add_trace(go.Scatter(x=[date_time], y=[0], mode='markers', marker=dict(size=df_filtered.loc[date_time, 'total_volume'] / 5000000, color='black'), text=hover_text, hoverinfo='text', showlegend=False))

# Set the layout
fig.update_layout(
    title="BTCUSDT Candlestick Chart with Liquidation Levels and Total Volume",
    xaxis_title="Date",
    yaxis_title="Volume",
    hovermode="x unified",
    width=1850,
    height=950,
    yaxis_range=[min(df['volume_true']), max(df['volume_false'])]  # Set y-axis range
)

# Show the plot
fig.show()


Label 1: 9 points found
Label 2: 6 points found
Label 3: 4 points found
Label 4: 1 points found
Label 5: 1 points found
Label 6: 0 points found
Label 7: 1 points found
Label 8: 0 points found
